In [3]:
import os

In [4]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml\\AI-powered-Bank-Product-Recommender-Chatbot\\research'

In [5]:
os.chdir("../.")

In [6]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml\\AI-powered-Bank-Product-Recommender-Chatbot'

In [7]:
project_dir = "C:/Users/RICH-FILES/Desktop/ml/AI-powered-Bank-Product-Recommender-Chatbot"
os.chdir(project_dir)

In [8]:
from dataclasses import dataclass
from pathlib import Path


    
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    local_data_file: Path
    export_csv_path: Path
    table_name: str
    
    

    

In [9]:
from BankProducts.constants import *
from BankProducts.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,    
        params_filepath = PARAMS_FILE_PATH,
        #schema_filepath: str = SCHEMA_FILE_PATH,
        ):
       
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        #self.schema = read_yaml(schema_filepath)        
    
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
    
    
        create_directories([self.config.artifacts_root])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            local_data_file=Path(config.local_data_file),
            export_csv_path=Path(config.export_csv_path),
            table_name=config.table_name
            
            
        )
        
        return data_ingestion_config

In [11]:
# Import necessary libraries
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
from BankProducts import logger
from dotenv import load_dotenv
import os
from BankProducts.utils.common import get_size
from sqlalchemy import text 

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
        self.engine = self._create_engine_from_env()

    def _create_engine_from_env(self):
        """Load DB credentials and return SQLAlchemy engine."""
        load_dotenv()
        db_user = os.getenv("DB_USER")
        db_password = os.getenv("DB_PASSWORD")
        db_host = os.getenv("DB_HOST")
        db_port = os.getenv("DB_PORT")
        db_name = os.getenv("DB_NAME")

        return create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

    def initiate_data_ingestion(self):
        """Load local CSV into PostgreSQL database."""
        logger.info("Starting data ingestion process...")

        # Check if file exists
        csv_path = self.config.local_data_file
        if not os.path.exists(csv_path):
            logger.error("CSV file not found. Please check the file path.")
            return

        try:
            df = pd.read_csv(csv_path)
            df.to_sql("bank_transactions", self.engine, if_exists="replace", index=False)
            logger.info("Dataset successfully loaded into PostgreSQL.")
        except Exception as e:
            logger.error(f"Failed to load data into PostgreSQL: {e}")

    def extract_and_save_data(self):
        """Fetch data from DB and save to CSV."""
        logger.info("Extracting and saving data...")

        query = "SELECT * FROM bank_transactions;"

        try:
            with self.engine.connect() as connection:
                result = connection.execute(text("SELECT to_regclass('bank_transactions')"))
                table_exists = result.scalar() is not None

            if not table_exists:
                logger.error("Table 'bank_transactions' does not exist in the database.")
                return

            logger.info("Fetching data from the 'bank_transactions' table...")
            df = pd.read_sql(query, self.engine)
            output_path = self.config.export_csv_path
            df.to_csv(output_path, index=False)
            logger.info(f"Data successfully exported to '{output_path}'.")

        except Exception as e:
            logger.error(f"Error while extracting data from database: {e}")


In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    #data_ingestion.initiate_data_ingestion()
    data_ingestion.extract_and_save_data()
    
except Exception as e:
    logger.exception(e)
    raise e


[2025-06-02 22:33:09,760: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-02 22:33:09,768: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-02 22:33:09,771: INFO: common: created directory at: artifacts]
[2025-06-02 22:33:09,772: INFO: common: created directory at: artifacts]
[2025-06-02 22:33:09,908: INFO: 1616883339: Extracting and saving data...]
[2025-06-02 22:33:10,124: INFO: 1616883339: Fetching data from the 'bank_transactions' table...]
[2025-06-02 22:33:10,628: INFO: 1616883339: Data successfully exported to 'artifacts\data_ingestion\customer_data.csv'.]


In [14]:
import pandas as pd
df = pd.read_csv(data_ingestion_config.export_csv_path)
print(df.head())

   transactionid  customerid transactiondate transactiontype        amount  \
0              1        8270      2025-01-29    Card Payment   6980.185223   
1              2        1860      2023-02-10         Deposit  10786.371850   
2              3        6390      2024-03-07        Transfer   3982.761111   
3              4        6191      2023-07-04      Withdrawal  12408.644140   
4              5        6734      2025-02-05             Fee   1868.260998   

    productcategory productsubcategory branchcity  branchlat  branchlong  \
0  Checking Account               Gold    Seville    37.3891     -5.9845   
1          Mortgage               Gold     Murcia    37.9847     -1.1287   
2              Loan           Platinum     Malaga    36.7213     -4.4214   
3          Mortgage           Standard    Seville    37.3891     -5.9845   
4  Checking Account           Platinum     Murcia    37.9847     -1.1287   

  channel currency  creditcardfees  insurancefees  latepaymentamount  \
0 